# PYTHON DB INTERACTION

In [2]:
clients_dir = "/Volumes/GoogleDrive/Shared drives/Buffalo Graphics/clients"


In [10]:
from Mongo import Client
import os


def sync_clients_dir(dir: str) -> None:
    # return early on invalid paths

    if os.path.isdir(dir) is False:
        print(f"{dir}\nis not a directory")
        return

    folder_clients = os.listdir(dir)
    db = Client().clients
    
    for folder_name in folder_clients:
        
        try:
            
            db.insert_one({
                "name": folder_name,
                "file_path": os.path.join(folder_name, dir),
                "files": []
            }).inserted_id

        except ValueError as e:
            print(e)
            
sync_clients_dir(clients_dir)            




DuplicateKeyError: E11000 duplicate key error collection: buffalographics.clients index: _name dup key: { name: "Bell Concrete" }, full error: {'index': 0, 'code': 11000, 'keyPattern': {'name': 1}, 'keyValue': {'name': 'Bell Concrete'}, 'errmsg': 'E11000 duplicate key error collection: buffalographics.clients index: _name dup key: { name: "Bell Concrete" }'}

In [ ]:
import os
from pprint import pprint

def clients_from_dir(dir: str) -> None:
    _clients = db["clients"]
    clients = []
    
    for c in _clients.find():
        clients.append(c.get("name"))
        
    inserted = []
    
    for client in os.listdir(dir):
        
        if _clients.find_one({"name": client}) is None:
    
            if client in clients:

                client_obj = {
                    "name": client,
                    "file_path": os.path.join(clients_dir, client)
                }

                client_id = _clients.insert_one(client_obj).inserted_id
                print(client_id)
                if client_id is not None:
                    inserted.append(
                        ({"_id": client_id, "name": client, }))

    pprint({
        "created": f"{len(inserted)} new clients",
        "client_count": _clients.count_documents({})
    })


clients_from_dir(clients_dir)


In [ ]:
def format_job_name(job_full_path, dir_name):
    name = job_full_path.replace(dir_name+'/', '')

    if name == "PRINT":
        name = "default"

    name = name.replace("/PRINT", '')
    
    return name

In [ ]:
def job_file_doc(job):
    print(job)
    _file = db["files"]
    file_names = glob(f"{job['dir']}/*.pdf")
    files = []
    for file in file_names: 
        files.append(create_file_obj(job["client_id"], job["_id"],file))
            
    if len(files) != 0:
        return _file.insert_many(files).inserted_ids

In [ ]:
def create_job_doc(obj, job_full_path):
    # obj = { "name": "Company Name" }
    dir = os.path.join(clients_dir, obj['name'])


    job = {
        "client_id": obj["_id"],
        "name": format_job_name(job_full_path, dir),
        "dir": job_full_path,
    }

    
    return job


In [ ]:
from Mongo import Client

from glob import glob
import os
def link_jobs(q):
    db = Client()['clients']
    _clients = db["clients"]
    _jobs = db["jobs"]

    client = _clients.find_one(q)
    dir_name = os.path.join(clients_dir, client['name'])

    jobs = []

    for job_full_path in glob(f"{dir_name}/**/**/PRINT", recursive=True):
        job = create_job_doc(client, job_full_path)
        jobs.append(_jobs.insert_one(job).inserted_id)

    return _clients.update_one(q, {"$set": {'jobs': jobs}}).upserted_id




In [ ]:
clients = []

for client in db["clients"].find():
    print(client)
    for job in db["jobs"].find({"client_id":client["_id"]}):
        if job.get("client_id",None)!=None:
            job_files = job_file_doc(job)
            db["jobs"].update_one(job,{"$set":{'files':job_files}})
    # if 'name' in c:
    #     print(link_jobs({'name': c['name']}))


{'_id': ObjectId('60a73232e7bd11f6121d32c3'), 'clients': [{'name': 'Phillips Sitework'}, {'name': 'NeBo Trucking Inc'}, {'name': 'The Hornet Stop'}, {'name': 'Wildcatter Redi-Mix'}, {'name': 'Pegasus Link Constructors'}, {'name': 'H&H Concrete'}, {'name': 'Preferred Materials'}, {'name': 'BMCR Concrete'}, {'name': 'Second Chance Boxing'}, {'name': 'H&S Excavation'}, {'name': 'Yamaha 3'}, {'name': 'C-CON Services'}, {'name': 'Raptor Concrete'}, {'name': 'Numo RMFG'}, {'name': 'Estrada Ready Mix Concrete'}, {'name': 'Bell Concrete'}, {'name': 'Titan Ready Mix'}, {'name': "Charley's Concrete"}, {'name': 'TAP Leasing'}]}
{'_id': ObjectId('60a7335d7877a34d71d940f7'), 'name': 'Bell Concrete', 'file_path': '/Volumes/GoogleDrive/Shared drives/Buffalo Graphics/clients/Bell Concrete'}
{'_id': ObjectId('60a7335d7877a34d71d940f8'), 'name': 'Titan Ready Mix', 'file_path': '/Volumes/GoogleDrive/Shared drives/Buffalo Graphics/clients/Titan Ready Mix'}
{'_id': ObjectId('60a7335d7877a34d71d940f9'), 'na

## DELETE DB

In [ ]:
from Mongo import Client
import os

db = Client()

db["clients"].delete_many({})
db["jobs"].delete_many({})
db["files"].delete_many({})
